In [256]:
# !pip3 install pandas

import os
import math
import pandas as pd
from IPython.display import display, HTML


In [274]:

def dfp(df):
    pd.options.display.float_format = '{:20,.0f}'.format
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)
    styles = [
        {'selector': 'tr', 'props': [('line-height', '12px')]},        # reduce row height
        {'selector': 'td, th', 'props': [('padding', '2px 20px'), ('font-size', '12px')]},  # reduce padding and font size
    ]
    styled_df = df.style.set_table_styles(styles).format('{:20,.2f}')
    html = f'<div style="height:500px; overflow:auto">{styled_df.to_html()}</div>'
    
    display(HTML(html))

def corpusGrowth_trends(age1=30, age2=60, unit=135000, inflation=0.04, realWageGrowth=0.06):
    rate_start = 0.10
    rate_interval = 0.01
    rate_end = 0.13
    n_rates = int((rate_end - rate_start) / rate_interval) + 1

    # Prepare column order: age, then for each rate group: rate%, corpus, annualised return, present year unit
    columns = ['age']
    for i in range(n_rates):
        r = rate_start + i * rate_interval
        percent = f"{r:.2%}"
        columns += [
            f'rate_{percent}',
            f'corpus_{percent}',
            f'present_year_unit_{percent}'
        ]

    data = []
    # Initialize first row (age, then zeros and None for values)
    row = [age1]
    for i in range(n_rates):
        r_percent = f"{rate_start + i*rate_interval:.2%}"
        row += [rate_start + i*rate_interval, 0, None]
    data.append(row)

    for year in range(1, age2 - age1):
        curr_row = [age1 + year]
        prev_corpus = [data[-1][1 + i * 3 + 1] for i in range(n_rates)]  # corpus cols offset by 1 for rate col
        for i in range(n_rates):
            year_start_corpus = prev_corpus[i]
            rate = rate_start + i * rate_interval
            monthly_fraction = math.pow(1+rate - inflation + realWageGrowth, 1 / 12) 
            annualised_percent =  (math.pow(monthly_fraction, 12) ) 
            present_year_unit = unit * math.pow(1 + realWageGrowth, year - 1)

            if (monthly_fraction - 1) == 0:
                corpus = year_start_corpus*(1+rate)
            else:
                corpus = year_start_corpus*(1+rate) + (present_year_unit*( (math.pow(monthly_fraction, 12)-1)/(monthly_fraction-1) ))
            
            curr_row += [
                annualised_percent,
                corpus,
                present_year_unit
            ]
        data.append(curr_row)

    df = pd.DataFrame(data, columns=columns)
    df.set_index('age', inplace=True)
    return df



In [275]:

dfp(corpusGrowth_trends(age1=30, age2=51, unit=135000, inflation=0.00, realWageGrowth=0.00))


,rate_10.00%,corpus_10.00%,present_year_unit_10.00%,rate_11.00%,corpus_11.00%,present_year_unit_11.00%,rate_12.00%,corpus_12.00%,present_year_unit_12.00%,rate_13.00%,corpus_13.00%,present_year_unit_13.00%
age,,,,,,,,,,,,
30,0.10,0.00,nan,0.11,0.00,nan,0.12,0.00,nan,0.13,0.00,nan
31,1.10,"1,692,972.44","135,000.00",1.11,"1,700,136.30","135,000.00",1.12,"1,707,277.22","135,000.00",1.13,"1,714,395.51","135,000.00"
32,1.10,"3,555,242.13","135,000.00",1.11,"3,587,287.59","135,000.00",1.12,"3,619,427.70","135,000.00",1.13,"3,651,662.44","135,000.00"
33,1.10,"5,603,738.79","135,000.00",1.11,"5,682,025.52","135,000.00",1.12,"5,761,036.24","135,000.00",1.13,"5,840,774.06","135,000.00"
34,1.10,"7,857,085.11","135,000.00",1.11,"8,007,184.62","135,000.00",1.12,"8,159,637.81","135,000.00",1.13,"8,314,470.20","135,000.00"
35,1.10,"10,335,766.06","135,000.00",1.11,"10,588,111.23","135,000.00",1.12,"10,846,071.56","135,000.00",1.13,"11,109,746.83","135,000.00"
36,1.10,"13,062,315.11","135,000.00",1.11,"13,452,939.76","135,000.00",1.12,"13,854,877.37","135,000.00",1.13,"14,268,409.43","135,000.00"
37,1.10,"16,061,519.06","135,000.00",1.11,"16,632,899.43","135,000.00",1.12,"17,224,739.87","135,000.00",1.13,"17,837,698.17","135,000.00"
38,1.10,"19,360,643.41","135,000.00",1.11,"20,162,654.67","135,000.00",1.12,"20,998,985.87","135,000.00",1.13,"21,870,994.44","135,000.00"
